In [ ]:
import argparse
import sys
import shap
import numpy as np
from transformers import AutoTokenizer

In [ ]:
def main(args, sampler, scorer):
    if args['randomize_data']:
        sampler.randomize()
    conv_to_explain = []
    human_scores = []
    for i, (conversation, conv_hscores) in enumerate(sampler.get_next_with_scores()):
        conv_to_explain.append(conversation)
        if conv_hscores:
            human_scores.append(conv_hscores)
        if not args['all'] and (i + 1) == args['num_examples']:
            break
        
    print(f'Number of conversation: {len(conv_to_explain)}')
    scores = None
    if not args['human_only']:
        scorer.set_aggregate(args['aggregate'])
        formatted_conversations = scorer.format_conversations(conv_to_explain)
        scores = scorer.get_scores(formatted_conversations)
    return scores, human_scores

In [ ]:
args = {'sampler_path' : '/home/ubuntu/fed/', \
        'scorer_path' : '/home/ubuntu/fed/', \
        'randomize_data': False, \
        'all' : True, \
        'num_examples' : 20, \
        'e' : 'vocab', \
        'top_k' : 1, \
        'aggregate' : False, \
        'human_only': False}

In [ ]:
sys.path.insert(0, args['scorer_path'])
if args['sampler_path'] != args['scorer_path']:
    sys.path.insert(0, args['sampler_path'])

In [ ]:
from sample import conversation_sampler
from score import conversation_scorer

sampler = conversation_sampler()
if args['human_only']:
    scorer = None
else:
    scorer = conversation_scorer()

In [ ]:
dataset_scores = main(args, sampler, scorer)

In [34]:
metric_scores = np.array(dataset_scores[0])
for i in range(dataset_scores[0].shape[0]):
    min_ = np.min(metric_scores[i, :])
    max_ = np.max(metric_scores[i, :])
    metric_scores[i, :] = (metric_scores[i, :] - min_)/(max_ - min_)
metric_scores.shape

(41, 18)

In [35]:
'Average:', metric_scores.mean(0), 'Standard Deviation:', metric_scores.std(0), metric_scores.std(0)/metric_scores.mean(0)

(array([3.62425447e-03, 2.29612133e-04, 9.00407101e-03, 9.88417488e-01,
        9.92608178e-01, 2.77419374e-02, 2.69933709e-02, 2.73536728e-02,
        9.86204995e-01, 9.81119943e-01, 9.99996887e-01, 9.97035981e-01,
        9.92082955e-01, 7.82766055e-03, 9.88867415e-01, 1.20772519e-02,
        7.95702998e-03, 4.98337737e-03]),
 array([2.35031236e-03, 8.26808370e-04, 4.16617560e-03, 5.93057858e-03,
        5.26173005e-03, 1.04417084e-02, 1.00573451e-02, 1.05340356e-02,
        5.22650491e-03, 9.31698053e-03, 1.96892468e-05, 1.86914122e-03,
        3.44252018e-03, 3.05230749e-03, 6.22528867e-03, 6.27523288e-03,
        3.48596020e-03, 3.48565386e-03]),
 array([6.48495404e-01, 3.60089147e+00, 4.62699106e-01, 6.00007451e-03,
        5.30091346e-03, 3.76387136e-01, 3.72585739e-01, 3.85104979e-01,
        5.29961311e-03, 9.49627066e-03, 1.96893081e-05, 1.87469786e-03,
        3.46999227e-03, 3.89938664e-01, 6.29537244e-03, 5.19591124e-01,
        4.38098161e-01, 6.99456132e-01]))

In [ ]:
# (array([-0.08528083, -0.11211523, -0.04663924,  7.71722919,  7.75530161,
#          0.09503976,  0.09051521,  0.0922358 ,  7.70058043,  7.66967944,
#          7.80357404,  7.77943006,  7.74207685, -0.05768314,  7.72689474,
#         -0.0226819 , -0.04903744, -0.07541956]),
#  array([0.04074469, 0.05115739, 0.02987215, 0.60181509, 0.59481855,
#         0.03631466, 0.03175073, 0.03627523, 0.60053694, 0.60838478,
#         0.58828052, 0.5928349 , 0.59737771, 0.04197221, 0.60055466,
#         0.01621715, 0.02788617, 0.0652111 ]),
#  array([-0.47777078, -0.45629292, -0.64049398,  0.07798331,  0.07669831,
#          0.38209963,  0.35077783,  0.393288  ,  0.07798593,  0.07932336,
#          0.07538604,  0.07620544,  0.07715988, -0.72763384,  0.07772264,
#         -0.71498187, -0.56867098, -0.86464442]))

In [36]:
categories = ['interesting', 'engaging', 'specific', 'relevant', 'correct', \
                'semantically appropriate', 'understandable', 'fluent', 'coherent', 'error recovery', \
                'consistent', 'diverse', 'depth', 'likeable', 'understandable', 'flexible', 'informative', 'inquisitive']
score_matrix = []
for conv_hscores in dataset_scores[1]:
    score_matrix.append([])
    for cat in categories:
        score_matrix[-1].append(conv_hscores[cat]/4)
np.array(score_matrix).mean(0), np.array(score_matrix).std(0), np.array(score_matrix).std(0)

(array([0.35416672, 0.38343284, 0.38392102, 0.42136018, 0.40837014,
        0.4303052 , 0.24055536, 0.43979353, 0.4695122 , 0.43882114,
        0.24390244, 0.44268293, 0.43292683, 0.46097561, 0.24055536,
        0.47317073, 0.44756098, 0.44512195]),
 array([0.06509743, 0.05196585, 0.05385796, 0.04617196, 0.04954895,
        0.04444899, 0.01188409, 0.03359151, 0.05282051, 0.09266108,
        0.02261858, 0.04627723, 0.06402149, 0.04206875, 0.01188409,
        0.03840979, 0.0562564 , 0.06028882]),
 array([0.06509743, 0.05196585, 0.05385796, 0.04617196, 0.04954895,
        0.04444899, 0.01188409, 0.03359151, 0.05282051, 0.09266108,
        0.02261858, 0.04627723, 0.06402149, 0.04206875, 0.01188409,
        0.03840979, 0.0562564 , 0.06028882]))